In [1]:
from utils import *
from model import *
import numpy as np
import torch
import sys
import torch.nn.functional as F
import torch.optim as optim
from utils_for_experiments import *
# from transform_wrappers_multiprocessing import *
from transform_wrappers import *
from matplotlib import pyplot as plt
from tqdm import tqdm
from discretize import *
from visualization import *


np.set_printoptions(linewidth=np.inf)

if __name__ == "__main__":

    # device = "cuda" if torch.cuda.is_available() else "cpu"
    device = "cpu"
    print(f"using device {device}")

    ####################     Generation parameters     #######################################################
    dataArgs = dict()

    maximum_number_of_nodes_n = "20" #@param [12, 24, 30, 48]
    dataArgs["max_n_node"] = int(maximum_number_of_nodes_n)

    range_of_linkage_probability_p = "0.0, 1.0" #@param [[0.0,1.0], [0.2,0.8], [0.5,0.5]]
    dataArgs["p_range"] = [float(range_of_linkage_probability_p.split(",")[0]), float(range_of_linkage_probability_p.split(",")[1])]

    node_attributes = "degree" #@param ["uniform", "degree", "random"]
    dataArgs["node_attr"] = node_attributes

    number_of_graph_instances = "5000" #@param [1, 100, 1000, 10000, 25000, 50000, 100000, 200000, 500000, 1000000]
    dataArgs["n_graph"] = int(number_of_graph_instances)

    dataArgs["upper_triangular"] = False
    A, Attr, Param, Topol = generate_data_v2(dataArgs)
    # g, a, attr = unpad_data(A[0], Attr[0])

    ####################     Model parameters     #######################################################
    modelArgs = {"gnn_filters": 2, "conv_filters": 16, "kernel_size": 3}

    number_of_latent_variables= "20" #@param [1, 2, 3, 4, 5]
    modelArgs["latent_dim"] = int(number_of_latent_variables)

    trainArgs = dict()

    weight_graph_reconstruction_loss = "30" #@param [0, 1, 2, 3, 5, 10, 20]
    weight_attribute_reconstruction_loss = "5" #@param [0, 1, 2, 3, 5, 10, 20]
    beta_value = "20" #@param [0, 1, 2, 3, 5, 10, 20]
    trainArgs["loss_weights"] = [int(weight_graph_reconstruction_loss), int(weight_attribute_reconstruction_loss), int(beta_value)]

    epochs = "35" #@param [10, 20, 50]
    trainArgs["epochs"] = int(epochs)
    batch_size = "512" #@param [2, 4, 8, 16, 32, 128, 512, 1024]
    trainArgs["batch_size"] = int(batch_size)
    early_stop = "2" #@param [1, 2, 3, 4, 10]
    trainArgs["early_stop"] = int(early_stop)
    train_test_split = "0.2" #@param [0.1, 0.2, 0.3, 0.5]
    train_validation_split = "0.1" #@param [0.1, 0.2, 0.3, 0.5]
    trainArgs["data_split"] = float(train_test_split)
    trainArgs["validation_split"] = float(train_validation_split)
    lr = "0.001"  #@param [0.1, 0.01, 0.001, 0.0001, 0.00001]
    trainArgs["lr"] = float(lr)



    ## Train and Test Split _______________________________________________

    A_train = torch.from_numpy(A[:int((1-trainArgs["data_split"]-trainArgs["validation_split"])*A.shape[0])])
    Attr_train = generate_batch(torch.from_numpy(Attr[:int((1-trainArgs["data_split"]-trainArgs["validation_split"])*Attr.shape[0])]), trainArgs["batch_size"])
    Param_train = generate_batch(torch.from_numpy(Param[:int((1-trainArgs["data_split"]-trainArgs["validation_split"])*Param.shape[0])]), trainArgs["batch_size"])
    Topol_train = generate_batch(torch.from_numpy(Topol[:int((1-trainArgs["data_split"]-trainArgs["validation_split"])*Topol.shape[0])]), trainArgs["batch_size"])

    A_validate = torch.from_numpy(A[int((1-trainArgs["data_split"]-trainArgs["validation_split"])*A.shape[0]):int((1-trainArgs["data_split"])*Attr.shape[0])])
    Attr_validate = generate_batch(torch.from_numpy(Attr[int((1-trainArgs["data_split"]-trainArgs["validation_split"])*Attr.shape[0]):int((1-trainArgs["data_split"])*Attr.shape[0])]), trainArgs["batch_size"])
    Param_validate = generate_batch(torch.from_numpy(Param[int((1-trainArgs["data_split"]-trainArgs["validation_split"])*Param.shape[0]):int((1-trainArgs["data_split"])*Attr.shape[0])]), trainArgs["batch_size"])
    Topol_validate = generate_batch(torch.from_numpy(Topol[int((1-trainArgs["data_split"]-trainArgs["validation_split"])*Topol.shape[0]):int((1-trainArgs["data_split"])*Attr.shape[0])]), trainArgs["batch_size"])

    A_test = torch.from_numpy(A[int((1-trainArgs["data_split"])*A.shape[0]):])
    Attr_test = generate_batch(torch.from_numpy(Attr[int((1-trainArgs["data_split"])*Attr.shape[0]):]), trainArgs["batch_size"])
    Param_test = generate_batch(torch.from_numpy(Param[int((1-trainArgs["data_split"])*Param.shape[0]):]), trainArgs["batch_size"])
    Topol_test = generate_batch(torch.from_numpy(Topol[int((1-trainArgs["data_split"])*Topol.shape[0]):]), trainArgs["batch_size"])


    # print(A_train.shape)
    # print(len(Attr_train), Attr_train[0].shape)

    ## build graph_conv_filters
    SYM_NORM = True
    A_train_mod = generate_batch(preprocess_adj_tensor_with_identity(torch.squeeze(A_train, -1), SYM_NORM), trainArgs["batch_size"])
    A_validate_mod = generate_batch(preprocess_adj_tensor_with_identity(torch.squeeze(A_validate, -1), SYM_NORM), trainArgs["batch_size"])
    A_test_mod = generate_batch(preprocess_adj_tensor_with_identity(torch.squeeze(A_test, -1), SYM_NORM), trainArgs["batch_size"])

    A_train = generate_batch(A_train, trainArgs["batch_size"])
    A_validate = generate_batch(A_validate, trainArgs["batch_size"])
    A_test = generate_batch(A_test, trainArgs["batch_size"])

    train_data = (Attr_train, A_train_mod, Param_train, Topol_train)
    validate_data = (Attr_validate, A_validate_mod, Param_validate, Topol_validate)
    test_data = (Attr_test, A_test_mod, Param_test, Topol_test)

    # attribute first -> (n, n), adjacency second -> (n, n, 1)
    modelArgs["input_shape"], modelArgs["output_shape"] = ((Attr_train[0].shape[1], Attr_train[0].shape[2]), (int(A_train_mod[0].shape[1] / modelArgs["gnn_filters"]), A_train_mod[0].shape[2], 1)),\
                                                          ((Attr_test[0].shape[1], Attr_test[0].shape[1]), (int(A_test_mod[0].shape[1] / modelArgs["gnn_filters"]), A_test_mod[0].shape[2], 1))
    # print(modelArgs["input_shape"], modelArgs["output_shape"])
    # print(A_train[0].shape)



  1%|▌                                                                              | 39/5000 [00:00<00:13, 365.46it/s]

using device cpu

============= Generating Data ===========================


100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:12<00:00, 385.38it/s]


done


In [2]:


######################      VAE        ##########################################
operation_name = "density"  ## ["transitivity", "density", "forest fire ..."]
# param_path = operation_name + "_pretrained" + "_" + maximum_number_of_nodes_n
param_path = "."
vae = torch.load(param_path + "/vae.model")

train_losses = []
validation_losses = []
batched_z = []
batched_A_hat = []
batched_Attr_hat = []
batched_A_hat_discretized = []
batched_A_hat_discretized_test = []
batched_gcn_filters_from_A_hat = []
batched_z_test = []
batched_A_hat_test = []
batched_Attr_hat_test = []
batched_gcn_filters_from_A_hat_test = []
batched_A_hat_raw_train = []
batched_A_hat_raw_test = []
batched_A_hat_max_train = []
batched_A_hat_max_test = []
batched_A_hat_min_train = []
batched_A_hat_min_test = []
print("\n\n =================Extracting useful information=====================")
vae.eval()

def index_of(my_list, target):
    try: return my_list.index(target)
    except: return dataArgs["max_n_node"]

for e in range(1):
    loss_cum = 0
    for i in range(len(Attr_train)):
        attr = Attr_train[i].float().to(device)
        A = A_train[i].float().to(device)
        graph_conv_filters = A_train_mod[i].float().to(device)

        z, z_mean, z_log_var, A_hat, attr_hat, A_hat_raw, max_score_per_node, min_score_per_node = vae(attr, graph_conv_filters)

        if e + 1 == 1:
            batched_z.append(z.detach())
            batched_Attr_hat.append(attr_hat.detach())
            batched_A_hat.append(A_hat.detach())
            temp = A_hat.detach().cpu()
            batched_gcn_filters_from_A_hat.append(preprocess_adj_tensor_with_identity(torch.squeeze(temp, -1), symmetric = False))

            A_discretize = A.cpu().squeeze().numpy()
            A_hat_discretize = A_hat.detach().cpu().squeeze().numpy()
            discretizer = Discretizer(A_discretize, A_hat_discretize)
            A_hat_discretize = discretizer.discretize('hard_threshold')
            A_hat_discretize = torch.unsqueeze(torch.from_numpy(A_hat_discretize), -1)

            batched_A_hat_discretized.append(A_hat_discretize)
            batched_A_hat_raw_train.append(A_hat_raw.detach())
            batched_A_hat_max_train.append(max_score_per_node.detach())
            batched_A_hat_min_train.append(min_score_per_node.detach())

            # count = 0
            for j in range(len(batched_A_hat_discretized[i])):
                temp = list(torch.diag(batched_A_hat_discretized[i][j].detach().reshape(dataArgs["max_n_node"], -1)))[::-1]
                pred_node_num = dataArgs["max_n_node"] - index_of(list(temp), 1)
                Param_train[i][j][-1] = pred_node_num  # predicted node num have ~96% acc
                true_node_num = int(Param_train[i][j][0])
                # print(pred_node_num)
                # print(true_node_num)

                # count += pred_node_num == true_node_num
            # print(f"node prediction accuracy : {count / len(batched_A_hat_discretized[i])}")

        loss = loss_func((A, attr), (A_hat, attr_hat), z_mean, z_log_var, trainArgs, modelArgs)
        loss_cum += loss.item()

    print("Model loss {} ".format(loss_cum / len(Attr_train)))


    loss_cum = 0
    for i in range(len(Attr_validate)):
        attr = Attr_validate[i].float().to(device)
        A = A_validate[i].float().to(device)
        graph_conv_filters = A_validate_mod[i].float().to(device)

        z, z_mean, z_log_var, A_hat, attr_hat, A_hat_raw, max_score_per_node, min_score_per_node = vae(attr, graph_conv_filters)
        loss = loss_func((A, attr), (A_hat, attr_hat), z_mean, z_log_var, trainArgs, modelArgs)
        loss_cum += loss.item()

        if e + 1 == 1:
            batched_z_test.append(z.detach())
            batched_Attr_hat_test.append(attr_hat.detach())
            batched_A_hat_test.append(A_hat.detach())
            temp = A_hat.detach().cpu()
            batched_gcn_filters_from_A_hat_test.append(preprocess_adj_tensor_with_identity(torch.squeeze(temp, -1), symmetric = False))

            A_discretize = A.cpu().squeeze().numpy()
            A_hat_discretize = A_hat.detach().cpu().squeeze().numpy()
            discretizer = Discretizer(A_discretize, A_hat_discretize)
            A_hat_discretize = discretizer.discretize('hard_threshold', threshold=0.7)
            A_hat_discretize = torch.unsqueeze(torch.from_numpy(A_hat_discretize), -1)

            batched_A_hat_discretized_test.append(A_hat_discretize)
            batched_A_hat_raw_test.append(A_hat_raw.detach())
            batched_A_hat_max_test.append(max_score_per_node.detach())
            batched_A_hat_min_test.append(min_score_per_node.detach())

    # print("At Epoch {}, validation loss {} ".format(e + 1, loss_cum / len(Attr_validate)))

a,p,r,f = compute_score_batched(A_train, batched_A_hat_discretized)
print(f"VAE performance:  \n Accuracy: {a},  \n Precision: {p},  \n Recall: {r},  \n F1 Score: {f}\n")

density_ori, diameter_ori, cluster_coef_ori, edges_ori, avg_degree_ori = topological_measure(A_train)
density_hat, diameter_hat, cluster_coef_hat, edges_hat, avg_degree_hat = topological_measure(batched_A_hat_discretized)
print(f"--- Truth topology (averaged) ---\n density: {density_ori} \n diameter: {diameter_ori} "
      f"\n clustering coefficient: {cluster_coef_ori} \n edges: {edges_ori} \n avgerage degree {avg_degree_ori}\n")
print(f"--- Reconstructed topology (averaged) ---\n density: {density_hat} \n diameter: {diameter_hat} "
      f"\n clustering coefficient: {cluster_coef_hat} \n edges: {edges_hat} \n avgerage degree {avg_degree_hat}\n")


################ Load Discriminator #####################################
discriminator = torch.load(param_path + "/discriminator.model")




 =================Extracting useful information=====================
Model loss 2380.786865234375 
VAE performance:  
 Accuracy: 0.9141899999999935,  
 Precision: 0.7439750526004668,  
 Recall: 0.8276554645262381,  
 F1 Score: 0.7835875048821409

--- Truth topology (averaged) ---
 density: 0.24532481203007417 
 diameter: -1.0 
 clustering coefficient: 0.27123789549619354 
 edges: 46.611714285714285 
 avgerage degree 0.24532481203007417

--- Reconstructed topology (averaged) ---
 density: 0.2893398496240597 
 diameter: -1.0 
 clustering coefficient: 0.40446849661076467 
 edges: 54.97457142857143 
 avgerage degree 0.2893398496240597



In [3]:

############################# Steering GAN   ####################################

## training tip: same batch, same alpha!

# w = torch.randn_like(batched_z[0][0], requires_grad=True).unsqueeze(0).to(device)
w = torch.load(param_path + "/w_density.pt")
a_w1 = torch.load(param_path + "/a_w1_density.pt")
a_w2 = torch.load(param_path + "/a_w2_density.pt")
a_b1 = torch.load(param_path + "/a_b1_density.pt")
a_b2 = torch.load(param_path + "/a_b2_density.pt")




### Initialize generator
generator = Decoder_v2(modelArgs, trainArgs, device).to(device)

decoder_weight = dict(vae.decoder.named_parameters())
generator_weight = dict(generator.named_parameters())
for k in generator_weight.keys():
    assert k in decoder_weight
    generator_weight[k] = decoder_weight[k]
generator.eval()


discriminator.eval()
## operation = "transitivity", "density", "node_count"

# transform = GraphTransform(dataArgs["max_n_node"], operation = operation_name, sigmoid = False)
transform = GraphTransform(dataArgs["max_n_node"], operation = operation_name, sigmoid = False)
w_epochs = 1  ### adjust epoch here!!!


In [4]:

loss_train = []
w_A_train = []
w_A_hat_train = []
w_edit_A_hat_train = []
w_gen_A_hat_train = []
gen_A_raw_train = []
gen_A_max_train = []
gen_A_min_train = []
masked_norm_A_hats = []

for e in range(w_epochs):
    for i in tqdm(range(len(batched_A_hat_discretized))):

        fil = batched_gcn_filters_from_A_hat[i].float().to(device)
        attr_hat = batched_Attr_hat[i].float().to(device)
        # A_hat = batched_A_hat[i].to(device)
        A_hat = batched_A_hat_discretized[i].to(device)
        A = A_train[i]
        z = batched_z[i].to(device)

        ## discretize
        # A = A_train[i].cpu().numpy().squeeze(-1)
        # A_hat = A_hat.cpu().numpy().squeeze(-1)
        # discretizer = Discretizer(A, A_hat)
        # A_hat = discretizer.discretize('hard_threshold')
        # A = torch.unsqueeze(torch.from_numpy(A), -1)
        # A_hat = torch.unsqueeze(torch.from_numpy(A_hat), -1)

#         _, alpha_edit = transform.get_train_alpha(A_hat)
        _, alpha_edit = 0, 0.35
        # alpha_gen = a_w2 * F.relu(a_w1 * alpha_edit + a_b1) + a_b2
        sign = -1 if alpha_edit < 0 else 1
        alpha_gen = sign * torch.log(torch.abs(torch.tensor(alpha_edit)))

        ## first get edit and D(edit(G(z)))
        
        
        edit_attr = attr_hat
        edit_A = transform.get_target_graph(alpha_edit, A_hat, list(Param_train[i][:,-1].type(torch.LongTensor)))  # replace this with the edit(G(z)) attr & filter! Expect do all graphs in batch in one step!!
        # print(alpha_edit, alpha_gen)

#         temp = edit_A.detach().cpu()
#         edit_fil = preprocess_adj_tensor_with_identity(torch.squeeze(temp, -1), symmetric = False).to(device)
#         feature_edit, _ = discriminator(edit_attr.float(), edit_fil.float())


        # Then get G(z + aw) and D(G(z + aw))
    
        gen_A, gen_attr, gen_A_raw, gen_A_max, gen_A_min = generator(z + alpha_gen * w)
#         temp = gen_A.detach().cpu()
#         gen_fil = preprocess_adj_tensor_with_identity(torch.squeeze(temp, -1), symmetric = False).to(device)
#         feature_gen, preds = discriminator(gen_attr.float(), gen_fil.float())
#         labels = torch.ones(edit_attr.shape[0]).to(device)



        if e + 1 == w_epochs:
            w_A_train.append(A)
            w_A_hat_train.append(A_hat)
            w_edit_A_hat_train.append(edit_A)
            w_gen_A_hat_train.append(gen_A.detach())
            gen_A_raw_train.append(gen_A_raw.detach())
            masked_norm_A = masked_normalization(gen_A_raw.detach(), Param_train[i])
            masked_norm_A_hats.append(masked_norm_A)
            gen_A_max_train.append(gen_A_max.detach())
            gen_A_min_train.append(gen_A_min.detach())

print("====================== G(z + aw) v.s. edit(G(z))  results =============================")
gen_A_discretized = debugDiscretizer(w_A_hat_train, w_edit_A_hat_train, gen_A_raw_train, gen_A_max_train, gen_A_min_train, w_gen_A_hat_train, masked_norm_A_hats, discretize_method="hard_threshold", printMatrix=False, abortPickle=True)


#debugDecoder(w_edit_A_hat_train, [], w_gen_A_hat_train, [], discretize_method="hard_threshold", printMatrix=True)
# drawGraph(w_A_train, w_A_hat_train, w_edit_A_hat_train, w_gen_A_hat_train)
# drawGraphSaveFigure(w_A_train, w_A_hat_train, w_edit_A_hat_train, w_gen_A_hat_train, clearImage=True)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.72it/s]


====================== G(z + aw) v.s. edit(G(z))  results =============================
accuracy: 0.8927507142857133
precision: 0.8855145540822386
recall: 0.6271413799285077
f1 score: 0.7342619123192717


In [5]:
density_ori, diameter_ori, cluster_coef_ori, edges_ori, avg_degree_ori = topological_measure(batched_A_hat_discretized) # G(z)
density_ed, diameter_ed, cluster_coef_ed, edges_ed, avg_degree_ed = topological_measure(w_edit_A_hat_train) # edit(G(z), edit_alpha)
density_hat, diameter_hat, cluster_coef_hat, edges_hat, avg_degree_hat = topological_measure([torch.from_numpy(gen_A_discretized)]) # G(z + alpaha * w)
print(f"--- Original topology (averaged) ---\n density: {density_ori} \n diameter: {diameter_ori} "
      f"\n clustering coefficient: {cluster_coef_ori} \n edges: {edges_ori} \n avgerage degree {avg_degree_ori}\n")

print(f"--- Edit topology (averaged) ---\n density: {density_ed} \n diameter: {diameter_ed} "
      f"\n clustering coefficient: {cluster_coef_ed} \n edges: {edges_ed} \n avgerage degree {avg_degree_ed}\n")

print(f"--- Reconstructed topology (averaged) ---\n density: {density_hat} \n diameter: {diameter_hat} "
      f"\n clustering coefficient: {cluster_coef_hat} \n edges: {edges_hat} \n avgerage degree {avg_degree_hat}\n")

--- Original topology (averaged) ---
 density: 0.23133233082706686 
 diameter: -1.0 
 clustering coefficient: 0.40446849661076467 
 edges: 43.95314285714286 
 avgerage degree 0.23133233082706683

--- Edit topology (averaged) ---
 density: 0.3704706766917278 
 diameter: -0.9988571428571429 
 clustering coefficient: 0.5145780875193894 
 edges: 70.38942857142857 
 avgerage degree 0.3704706766917278

--- Reconstructed topology (averaged) ---
 density: 0.2622511278195479 
 diameter: -1.0 
 clustering coefficient: 0.3949161427834552 
 edges: 49.827714285714286 
 avgerage degree 0.2622511278195479



In [140]:
global_norm_A = []
global_edit_A = []
for idx,alpha in enumerate([0.01, 0.1, 0.2, 0.3, 0.4, 0.5]):
    print(f"!!!!!!!!!!!!!!==========================edit_alpha = {alpha} ===================================!!!!!!!!!!!!!!")
    loss_train = []
    w_A_train = []
    w_A_hat_train = []
    w_edit_A_hat_train = []
    w_gen_A_hat_train = []
    gen_A_raw_train = []
    gen_A_max_train = []
    gen_A_min_train = []
    masked_norm_A_hats = []
    threshold = 0.5 - 0.05 * (idx + 1)
    print(f"Current discretize threshold: {threshold}")
    for e in range(w_epochs):
        for i in tqdm(range(len(batched_A_hat_discretized))):

            fil = batched_gcn_filters_from_A_hat[i].float().to(device)
            attr_hat = batched_Attr_hat[i].float().to(device)
            # A_hat = batched_A_hat[i].to(device)
            A_hat = batched_A_hat_discretized[i].to(device)
            A = A_train[i]
            z = batched_z[i].to(device)

            _, alpha_edit = 0, alpha
        
            sign = -1 if alpha_edit < 0 else 1
            alpha_gen = sign * torch.log(torch.abs(torch.tensor(alpha_edit)))
            print(alpha_gen)

            edit_attr = attr_hat
            edit_A = transform.get_target_graph(alpha_edit, A_hat, list(Param_train[i][:,-1].type(torch.LongTensor)))  # replace this with the edit(G(z)) attr & filter! Expect do all graphs in batch in one step!!

            gen_A, gen_attr, gen_A_raw, gen_A_max, gen_A_min = generator(z + alpha_gen * w)

            if e + 1 == w_epochs:
                w_A_train.append(A)
                w_A_hat_train.append(A_hat)
                w_edit_A_hat_train.append(edit_A)
                w_gen_A_hat_train.append(gen_A.detach())
                gen_A_raw_train.append(gen_A_raw.detach())
                masked_norm_A = masked_normalization(gen_A_raw.detach(), Param_train[i])
                masked_norm_A_hats.append(masked_norm_A)
                print(f"Avaerge probability: {torch.mean(masked_norm_A[masked_norm_A > 1e-3])}")
                print(f"Avaerge probability variance: {torch.var(masked_norm_A[masked_norm_A > 1e-3])}")
                gen_A_max_train.append(gen_A_max.detach())
                gen_A_min_train.append(gen_A_min.detach())
                
        global_norm_A.append(masked_norm_A_hats)
        global_edit_A.append(w_edit_A_hat_train)


    print("====================== G(z + aw) v.s. edit(G(z))  results =============================")
    gen_A_discretized = debugDiscretizer(w_A_hat_train, w_edit_A_hat_train, gen_A_raw_train, gen_A_max_train, gen_A_min_train, w_gen_A_hat_train, masked_norm_A_hats, discretize_method="hard_threshold", printMatrix=False, abortPickle=True, threshold=threshold)

    density_ori, diameter_ori, cluster_coef_ori, edges_ori, avg_degree_ori = topological_measure(batched_A_hat_discretized) # G(z)
    density_ed, diameter_ed, cluster_coef_ed, edges_ed, avg_degree_ed = topological_measure(w_edit_A_hat_train) # edit(G(z), edit_alpha)
    density_hat, diameter_hat, cluster_coef_hat, edges_hat, avg_degree_hat = topological_measure([torch.from_numpy(gen_A_discretized)]) # G(z + alpaha * w)
    print(f"--- Original topology (averaged) ---\n density: {density_ori} \n diameter: {diameter_ori} "
          f"\n clustering coefficient: {cluster_coef_ori} \n edges: {edges_ori} \n avgerage degree {avg_degree_ori}\n")

    print(f"--- Edit topology (averaged) ---\n density: {density_ed} \n diameter: {diameter_ed} "
          f"\n clustering coefficient: {cluster_coef_ed} \n edges: {edges_ed} \n avgerage degree {avg_degree_ed}\n")

    print(f"--- Reconstructed topology (averaged) ---\n density: {density_hat} \n diameter: {diameter_hat} "
          f"\n clustering coefficient: {cluster_coef_hat} \n edges: {edges_hat} \n avgerage degree {avg_degree_hat}\n")
    print("\n\n")


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

!!!!!!!!!!!!!!==========================edit_alpha = 0.01 ===================================!!!!!!!!!!!!!!
Current discretize threshold: 0.45
tensor(-4.6052)


 14%|████████████                                                                        | 1/7 [00:00<00:02,  2.12it/s]

Avaerge probability: 0.4975826144218445
Avaerge probability variance: 0.04734661057591438
tensor(-4.6052)


 29%|████████████████████████                                                            | 2/7 [00:01<00:02,  1.99it/s]

...

 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [00:04<00:00,  1.54it/s]

Avaerge probability: 0.5156394839286804
Avaerge probability variance: 0.05410677567124367
tensor(-0.6931)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.54it/s]

Avaerge probability: 0.48545169830322266
Avaerge probability variance: 0.052768439054489136
====================== G(z + aw) v.s. edit(G(z))  results =============================


accuracy: 0.9525035714285667
precision: 0.9201901681677563
recall: 0.8987107225817093
f1 score: 0.9093236197227607
--- Original topology (averaged) ---
 density: 0.23133233082706686 
 diameter: -1.0 
 clustering coefficient: 0.40446849661076467 
 edges: 43.95314285714286 
 avgerage degree 0.23133233082706683

--- Edit topology (averaged) ---
 density: 0.3948075187969905 
 diameter: -0.9991428571428571 
 clustering coefficient: 0.5399600073358988 
 edges: 75.01342857142858 
 avgerage degree 0.3948075187969905

--- Reconstructed topology (averaged) ---
 density: 0.3837699248120277 
 diameter: -1.0 
 clustering coefficient: 0.5285956397360818 
 edges: 72.91628571428572 
 avgerage degree 0.3837699248120277






In [12]:
torch.nonzero(masked_norm_A)

..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


tensor([[  0,   0,   0],
        [  0,   0,   1],
        [  0,   0,   2],
        ...,
        [427,   5,   3],
        [427,   5,   4],
        [427,   5,   5]])

In [18]:
torch.mean(masked_norm_A[masked_norm_A > 1e-3])

tensor(0.4855)

In [141]:
def compare(i1,i2,i3):
    print(global_norm_A[i1][i2][i3].numpy())
    print(global_edit_A[i1][i2][i3].squeeze(-1).numpy().astype(int))

In [205]:
compare(0,0,449)

[[0.29 0.25 0.16 0.19 0.27 0.19 0.09 0.09 0.28 0.16 0.10 0.03 0.14 0.09 0.10 0.00 0.00 0.00 0.00 0.00]
 [0.25 0.35 0.14 0.20 0.18 0.09 0.03 0.07 0.37 0.09 0.06 0.00 0.19 0.17 0.13 0.00 0.00 0.00 0.00 0.00]
 [0.16 0.14 0.22 0.14 0.20 0.25 0.11 0.15 0.19 0.14 0.18 0.03 0.04 0.08 0.19 0.00 0.00 0.00 0.00 0.00]
 [0.19 0.20 0.14 0.22 0.17 0.14 0.05 0.16 0.27 0.10 0.12 0.02 0.16 0.16 0.21 0.00 0.00 0.00 0.00 0.00]
 [0.27 0.18 0.20 0.17 0.42 0.36 0.19 0.14 0.28 0.32 0.23 0.07 0.10 0.19 0.23 0.00 0.00 0.00 0.00 0.00]
 [0.19 0.09 0.25 0.14 0.36 0.45 0.23 0.22 0.19 0.30 0.32 0.09 0.02 0.15 0.29 0.00 0.00 0.00 0.00 0.00]
 [0.09 0.03 0.11 0.05 0.19 0.23 0.15 0.10 0.08 0.17 0.15 0.06 0.01 0.06 0.12 0.00 0.00 0.00 0.00 0.00]
 [0.09 0.07 0.15 0.16 0.14 0.22 0.10 0.25 0.14 0.12 0.25 0.05 0.08 0.17 0.34 0.00 0.00 0.00 0.00 0.00]
 [0.28 0.37 0.19 0.27 0.28 0.19 0.08 0.14 0.56 0.19 0.17 0.04 0.24 0.36 0.36 0.00 0.00 0.00 0.00 0.00]
 [0.16 0.09 0.14 0.10 0.32 0.30 0.17 0.12 0.19 0.30 0.22 0.07 0.04 0.18 0

In [130]:
alpha_gen

tensor(-0.6931)

In [82]:
np.set_printoptions(linewidth=150)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})